## Test I/O with networkx

In [1]:
import networkx as nx
from graphs import from_networkx, Graph

# 从 networkx 创建
nx_g = nx.karate_club_graph()
jax_g = from_networkx(nx_g)

print("从 NetworkX 创建的图:")
print(f"节点数: {jax_g.n_nodes}")
print(f"边数: {jax_g.n_edges}")
print("邻接矩阵 (前5x5):")
# 修改: 调用 to_adjacency_matrix() 方法
print(jax_g.to_adjacency_matrix())

从 NetworkX 创建的图:
节点数: 34
边数: 156
邻接矩阵 (前5x5):
[[0. 4. 5. ... 2. 0. 0.]
 [4. 0. 6. ... 0. 0. 0.]
 [5. 6. 0. ... 0. 2. 0.]
 ...
 [2. 0. 0. ... 0. 4. 4.]
 [0. 0. 2. ... 4. 0. 5.]
 [0. 0. 0. ... 4. 5. 0.]]


In [2]:
import networkx as nx
from graphs import from_networkx

nx_g = nx.karate_club_graph()
# 显式指定 'club' 作为特征键
jax_g = from_networkx(nx_g, node_feature_key='club') 

print(jax_g.node_features.T) # 打印特征

[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1]]


/Users/makotoxu/Documents/Coding/JAX/graph-jax/graphs/io.py:63: UserWarning: 字符串特征 'club' 被自动编码为整数: {'Mr. Hi': 0, 'Officer': 1}
  warnings.warn(


In [3]:
# --- 补充测试：有向图和带权重的边 ---
import numpy as np
import networkx as nx
from graphs import from_networkx, to_networkx

# 1. 创建一个复杂的有向、带权重的图
nx_g_directed = nx.DiGraph()
nx_g_directed.add_weighted_edges_from([
    (0, 1, 0.5), 
    (1, 2, 1.5), 
    (2, 0, 2.5),
    (2, 1, 3.0) # 反向边
])
# 添加一个孤立节点
nx_g_directed.add_node(3) 
# 为节点添加特征
features = {0: 10, 1: 20, 2: 30, 3: 40}
nx.set_node_attributes(nx_g_directed, features, 'value')

# 2. 转换到 JAX 图再转换回来
jax_g_directed = from_networkx(nx_g_directed, node_feature_key='value')
nx_g_reverted_directed = to_networkx(jax_g_directed, node_feature_key='value', create_using=nx.DiGraph)

# --- 验证 ---
print("--- 有向图转换验证 ---")

# 验证节点和边的数量
assert nx_g_directed.number_of_nodes() == nx_g_reverted_directed.number_of_nodes()
assert nx_g_directed.number_of_edges() == nx_g_reverted_directed.number_of_edges()
print("节点和边的数量一致。")

# 验证邻接矩阵
adj_original = nx.to_numpy_array(nx_g_directed)
# 修改: 调用 to_adjacency_matrix() 方法
is_adj_same = np.allclose(jax_g_directed.to_adjacency_matrix(), adj_original)
assert is_adj_same
print("邻接矩阵一致。")

# 验证节点特征
reverted_features_raw = nx.get_node_attributes(nx_g_reverted_directed, 'value')
reverted_features = np.array([reverted_features_raw[n] for n in sorted(reverted_features_raw.keys())])
assert np.array_equal(jax_g_directed.node_features.flatten(), reverted_features)
print("节点特征一致。")

# 验证边的权重和方向
original_edges = set((u, v, d.get('weight', 1.0)) for u, v, d in nx_g_directed.edges(data=True))
reverted_edges = set((u, v, d.get('weight', 1.0)) for u, v, d in nx_g_reverted_directed.edges(data=True))
assert original_edges == reverted_edges
print("边的方向和权重一致。")

print("\n补充测试完成，所有断言均通过。")

--- 有向图转换验证 ---
节点和边的数量一致。
邻接矩阵一致。
节点特征一致。
边的方向和权重一致。

补充测试完成，所有断言均通过。


## Test I/O with CSV & Json

In [4]:
# --- JSON 和 CSV 读写测试 ---
import os
import jax.numpy as jnp
import numpy as np
from graphs import Graph, to_json, from_json, to_csv, from_csv

# 1. 修改: 创建一个用于测试的稀疏 JAX 图
senders = jnp.array([0, 1, 1, 2])
receivers = jnp.array([1, 0, 2, 1])
weights = jnp.array([1.5, 1.5, 2.0, 2.0])
test_features = jnp.array([
    [1.0, 0.0],
    [0.0, 1.0],
    [1.0, 1.0],
])
# 修改: 使用新的稀疏构造函数
jax_g_test = Graph(
    senders=senders, 
    receivers=receivers, 
    edge_weights=weights,
    n_nodes=3, 
    n_edges=len(senders),
    node_features=test_features
)

# 定义文件名
json_path = "test_graph.json"
csv_path = "test_graph.csv"

# --- JSON 往返测试 ---
print("--- JSON 读写测试 ---")
to_json(jax_g_test, json_path)
# 修改: from_json 不再需要 directed 参数
reloaded_g_json = from_json(json_path)

# 验证 JSON
assert jax_g_test.n_nodes == reloaded_g_json.n_nodes
# 修改: 调用 to_adjacency_matrix() 方法
assert np.allclose(jax_g_test.to_adjacency_matrix(), reloaded_g_json.to_adjacency_matrix())
assert np.allclose(jax_g_test.node_features, reloaded_g_json.node_features)
print(f"JSON 读写验证成功。文件已保存到: {os.path.abspath(json_path)}")
# os.remove(json_path)

# --- CSV 往返测试 ---
print("\n--- CSV 读写测试 ---")
to_csv(jax_g_test, csv_path)
# 修改: from_csv 不再需要 directed 参数
reloaded_g_csv = from_csv(csv_path)

# 验证 CSV
assert jax_g_test.n_nodes == reloaded_g_csv.n_nodes
# 修改: 调用 to_adjacency_matrix() 方法
assert np.allclose(jax_g_test.to_adjacency_matrix(), reloaded_g_csv.to_adjacency_matrix())
print(f"CSV 读写验证成功。文件已保存到: {os.path.abspath(csv_path)}")
# os.remove(csv_path)

print("\nJSON 和 CSV 读写测试完成。")

--- JSON 读写测试 ---
JSON 读写验证成功。文件已保存到: /Users/makotoxu/Documents/Coding/JAX/graph-jax/test_graph.json

--- CSV 读写测试 ---
CSV 读写验证成功。文件已保存到: /Users/makotoxu/Documents/Coding/JAX/graph-jax/test_graph.csv

JSON 和 CSV 读写测试完成。


## Test for Matrix computation

In [5]:
# --- 内核函数测试 ---
from kernels import degree_matrix, laplacian_matrix, normalized_laplacian_sym

# 使用之前从 karate_club_graph 创建的 jax_g 对象
# 如果 jax_g 不存在，请先运行前面的单元格
print("--- 内核函数测试 ---")

# 1. 测试度矩阵
deg_matrix = degree_matrix(jax_g)
print("度矩阵 (前5x5):")
print(deg_matrix[:5, :5])

# 2. 测试拉普拉斯矩阵
lap_matrix = laplacian_matrix(jax_g)
print("\n拉普拉斯矩阵 (前5x5):")
print(lap_matrix[:5, :5])

# 3. 测试对称归一化拉普拉斯矩阵
norm_lap_sym = normalized_laplacian_sym(jax_g)
print("\n对称归一化拉普拉斯矩阵 (前5x5):")
print(norm_lap_sym[:5, :5])

--- 内核函数测试 ---
度矩阵 (前5x5):
[[16.  0.  0.  0.  0.]
 [ 0.  9.  0.  0.  0.]
 [ 0.  0. 10.  0.  0.]
 [ 0.  0.  0.  6.  0.]
 [ 0.  0.  0.  0.  3.]]

拉普拉斯矩阵 (前5x5):
[[42. -4. -5. -3. -3.]
 [-4. 29. -6. -3.  0.]
 [-5. -6. 33. -3.  0.]
 [-3. -3. -3. 18.  0.]
 [-3.  0.  0.  0.  8.]]

对称归一化拉普拉斯矩阵 (前5x5):
[[ 0.9375     -0.08333334 -0.07905694 -0.10206207 -0.14433756]
 [-0.08333334  0.8888889  -0.10540926 -0.13608277  0.        ]
 [-0.07905694 -0.10540926  0.9        -0.12909944  0.        ]
 [-0.10206207 -0.13608277 -0.12909944  0.8333334   0.        ]
 [-0.14433756  0.          0.          0.          0.6666667 ]]


In [6]:
# --- 内核函数验证 (与 NetworkX 对比) ---
import numpy as np
import networkx as nx
from kernels import degree_matrix, laplacian_matrix, normalized_laplacian_sym

# 确保 nx_g 和 jax_g 变量存在
print("--- 内核函数验证 (与 NetworkX 对比) ---\n")

# 1. 验证度矩阵
print("--- 1. 验证度矩阵 ---")
# NetworkX 计算方式:
nx_degrees = np.array([d for n, d in nx.degree(nx_g)])
nx_deg_matrix = np.diag(nx_degrees)
# 我们的计算方式:
jax_deg_matrix = degree_matrix(jax_g)

print("NetworkX 度矩阵 (前5x5):\n", nx_deg_matrix[:5, :5])
print("\n我们的 度矩阵 (前5x5):\n", np.asarray(jax_deg_matrix)[:5, :5])
print(f"\n--> 度矩阵是否与 NetworkX 一致: {np.allclose(nx_deg_matrix, jax_deg_matrix)}\n")
print("-" * 40)


# 2. 验证拉普拉斯矩阵
print("--- 2. 验证拉普拉斯矩阵 ---")
# NetworkX 计算方式:
nx_lap_matrix = nx.laplacian_matrix(nx_g).toarray()
# 我们的计算方式:
jax_lap_matrix = laplacian_matrix(jax_g)

print("NetworkX 拉普拉斯矩阵 (前5x5):\n", nx_lap_matrix[:5, :5])
print("\n我们的 拉普拉斯矩阵 (前5x5):\n", np.asarray(jax_lap_matrix)[:5, :5])
print(f"\n--> 拉普拉斯矩阵是否与 NetworkX 一致: {np.allclose(nx_lap_matrix, jax_lap_matrix)}\n")
print("-" * 40)


# 3. 验证对称归一化拉普拉斯矩阵
print("--- 3. 验证对称归一化拉普拉斯矩阵 ---")
# NetworkX 计算方式:
nx_norm_lap_matrix = nx.normalized_laplacian_matrix(nx_g).toarray()
# 我们的计算方式 (注意：为了与 networkx 标准行为对齐，需设置 add_self_loops=False)
jax_norm_lap_matrix = normalized_laplacian_sym(jax_g, add_self_loops=False)

print("NetworkX 归一化拉普拉斯矩阵 (前5x5):\n", nx_norm_lap_matrix[:5, :5])
print("\n我们的 归一化拉普拉斯矩阵 (前5x5):\n", np.asarray(jax_norm_lap_matrix)[:5, :5])
print(f"\n--> 对称归一化拉普拉斯矩阵是否与 NetworkX 一致: {np.allclose(nx_norm_lap_matrix, jax_norm_lap_matrix)}\n")

--- 内核函数验证 (与 NetworkX 对比) ---

--- 1. 验证度矩阵 ---
NetworkX 度矩阵 (前5x5):
 [[16  0  0  0  0]
 [ 0  9  0  0  0]
 [ 0  0 10  0  0]
 [ 0  0  0  6  0]
 [ 0  0  0  0  3]]

我们的 度矩阵 (前5x5):
 [[16.  0.  0.  0.  0.]
 [ 0.  9.  0.  0.  0.]
 [ 0.  0. 10.  0.  0.]
 [ 0.  0.  0.  6.  0.]
 [ 0.  0.  0.  0.  3.]]

--> 度矩阵是否与 NetworkX 一致: True

----------------------------------------
--- 2. 验证拉普拉斯矩阵 ---
NetworkX 拉普拉斯矩阵 (前5x5):
 [[42 -4 -5 -3 -3]
 [-4 29 -6 -3  0]
 [-5 -6 33 -3  0]
 [-3 -3 -3 18  0]
 [-3  0  0  0  8]]

我们的 拉普拉斯矩阵 (前5x5):
 [[42. -4. -5. -3. -3.]
 [-4. 29. -6. -3.  0.]
 [-5. -6. 33. -3.  0.]
 [-3. -3. -3. 18.  0.]
 [-3.  0.  0.  0.  8.]]

--> 拉普拉斯矩阵是否与 NetworkX 一致: True

----------------------------------------
--- 3. 验证对称归一化拉普拉斯矩阵 ---
NetworkX 归一化拉普拉斯矩阵 (前5x5):
 [[ 1.         -0.11461365 -0.13430383 -0.10910895 -0.16366342]
 [-0.11461365  1.         -0.19395246 -0.13130643  0.        ]
 [-0.13430383 -0.19395246  1.         -0.12309149  0.        ]
 [-0.10910895 -0.13130643 -0.12309149  1. 

In [ ]:
import os
# 这个标志告诉 JAX/XLA 在 CPU 上创建指定数量的“虚拟”设备。
# 将 '8' 替换为您希望使用的CPU核心数。
# 注意：这个单元格必须在任何 jax 导入之前运行。
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=8' 

# --- 高级内核与性能测试 ---
import timeit
import numpy as np
import networkx as nx
import jax
from graphs import from_networkx, Graph
import jax.numpy as jnp
from kernels import (
    random_walk_normalized_laplacian, 
    laplacian_eigensystem,
    spgemm,
    normalized_laplacian_sym
)


jax.config.update("jax_enable_x64", True)

# more heavy duty
# 使用 Barabasi-Albert 模型生成一个规模更大的图
n_nodes = 2500  # 节点数
m_edges = 4      # 每个新节点连接到的现有节点数
print(f"正在生成一个包含 {n_nodes} 个节点和大约 {n_nodes * m_edges} 条边的 Barabasi-Albert 图...")

# 1. 创建 NetworkX 图对象
nx_g = nx.barabasi_albert_graph(n=n_nodes, m=m_edges, seed=42)
print("NetworkX 图已生成.")
print(f"节点数: {nx_g.number_of_nodes()}, 边数: {nx_g.number_of_edges()}")


# 2. 将 NetworkX 图转换为我们的 JAX Graph 对象
# 假设 Graph.from_networkx 是正确的转换函数
jax_g = from_networkx(nx_g)
print("JAX Graph 对象已创建.\n")


print("--- 高级内核与性能测试 ---\n")

# --- 1. 随机游走归一化拉普拉斯矩阵 ---
print("--- 1. 验证随机游走归一化拉普拉斯矩阵 ---")

# NetworkX/NumPy 计算方式
def lrw_numpy(A):
    D = np.diag(np.sum(A, axis=1))
    # 防止除以零
    D_inv = np.linalg.inv(D)
    return np.eye(A.shape[0]) - D_inv @ A

A_nx = nx.to_numpy_array(nx_g)
lrw_nx_result = lrw_numpy(A_nx)

# 我们的计算方式
# 注意：为了与 networkx 标准行为对齐，需设置 add_self_loops=False
lrw_jax_result = random_walk_normalized_laplacian(jax_g, add_self_loops=False)

print("NumPy/NetworkX 结果 (前5x5):\n", lrw_nx_result[:5, :5])
print("\nJAX 结果 (前5x5):\n", lrw_jax_result[:5, :5])
print(f"\n结果是否一致: {np.allclose(lrw_nx_result, lrw_jax_result)}")

# 性能比较 (不含编译时间)
# 预热：先运行一次 JAX 函数以完成 JIT 编译
lrw_jax_result.block_until_ready() 

jax_time = timeit.timeit(lambda: random_walk_normalized_laplacian(jax_g, add_self_loops=False).block_until_ready(), number=100)
numpy_time = timeit.timeit(lambda: lrw_numpy(A_nx), number=100)

print(f"JAX 实现速度: {jax_time:.6f}s")
print(f"NumPy 实现速度: {numpy_time:.6f}s")
print("-" * 40)


# --- 2. 拉普拉斯特征分解 ---
print("--- 2. 验证拉普拉斯特征分解 ---")
k = 5 # 计算前 5 个特征值/向量

# 1. 首先用我们的函数计算对称归一化拉普拉斯矩阵
l_sym_jax = normalized_laplacian_sym(jax_g, add_self_loops=False, use_weights=False)

# 2. 然后对这个矩阵进行标准的 NumPy/SciPy 特征分解
# 确保 jax_enable_x64=True，这样 np.linalg.eigh 和我们的函数都在 float64 上操作
vals_ref, vecs_ref = np.linalg.eigh(np.asarray(l_sym_jax))
vals_ref, vecs_ref = vals_ref[:k], vecs_ref[:, :k]

# 3. 调用我们自己的 laplacian_eigensystem 函数
# 强制使用 float64 以便与 NumPy 的 eigh 进行精确比较
vals_jax, vecs_jax = laplacian_eigensystem(jax_g, k=k, use_weights=False, force_float64=True)

print("NumPy/SciPy 特征值 (基于我们的L_sym):\n", vals_ref)
print("\nJAX 特征值:\n", vals_jax)
print("\nNumPy/SciPy 特征向量 (前5行, 基于我们的L_sym):\n", vecs_ref[:5, :])
print("\nJAX 特征向量 (前5行):\n", vecs_jax[:5, :])

# 特征向量的符号可能不一致，但方向应一致，故比较绝对值
print(f"\n特征值是否一致 (atol=1e-5): {np.allclose(vals_ref, vals_jax, atol=1e-5)}")
print(f"特征向量是否一致 (atol=1e-5): {np.allclose(np.abs(vecs_ref), np.abs(vecs_jax), atol=1e-5)}")

# 性能比较
# 预热
vals_jax.block_until_ready()
vecs_jax.block_until_ready()

# 修正: 定义 NumPy 的基准矩阵 L_sym_nx，确保它对应的是大的 Barabasi-Albert 图
# 注意：这里我们使用 NumPy 的原生方式来计算，以进行公平的速度比较
# 而不是使用我们之前计算的 l_sym_jax，因为那会包含 JAX -> NumPy 的转换开销
A_nx = nx.to_numpy_array(nx_g)
L_sym_nx = nx.normalized_laplacian_matrix(nx_g).toarray()


jax_time = timeit.timeit(lambda: laplacian_eigensystem(jax_g, k=k, force_float64=True)[0].block_until_ready(), number=10)
numpy_time = timeit.timeit(lambda: np.linalg.eigh(L_sym_nx), number=10)

print(f"\nJAX 实现速度 (k={k}): {jax_time:.6f}s")
print(f"NumPy 实现速度: {numpy_time:.6f}s")
print("-" * 40)

正在生成一个包含 2500 个节点和大约 10000 条边的 Barabasi-Albert 图...
NetworkX 图已生成.
节点数: 2500, 边数: 9984
JAX Graph 对象已创建.

--- 高级内核与性能测试 ---

--- 1. 验证随机游走归一化拉普拉斯矩阵 ---
NumPy/NetworkX 结果 (前5x5):
 [[ 1.         -0.00515464 -0.00515464 -0.00515464 -0.00515464]
 [-0.01587302  1.          0.          0.          0.        ]
 [-0.03030303  0.          1.          0.          0.        ]
 [-0.01612903  0.          0.          1.          0.        ]
 [-0.03333333  0.          0.          0.          1.        ]]

JAX 结果 (前5x5):
 [[ 1.         -0.00515464 -0.00515464 -0.00515464 -0.00515464]
 [-0.01587302  1.          0.          0.          0.        ]
 [-0.03030303  0.          1.          0.          0.        ]
 [-0.01612903  0.          0.          1.          0.        ]
 [-0.03333334  0.          0.          0.          1.        ]]

结果是否一致: True
JAX 实现速度: 5.310564s
NumPy 实现速度: 41.733778s
----------------------------------------
--- 2. 验证拉普拉斯特征分解 ---
NumPy/SciPy 特征值 (基于我们的L_sym):
 [-3.34413108e-09  3.58

In [1]:
import os
# 这个标志告诉 JAX/XLA 在 CPU 上创建指定数量的“虚拟”设备。
# 注意：这个单元格必须在任何 jax 导入之前运行。
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=8' 

# --- 高级内核与性能测试 ---
import timeit
import numpy as np
import networkx as nx
import jax
from graphs import from_networkx, Graph
import jax.numpy as jnp
from kernels import (
    random_walk_normalized_laplacian, 
    laplacian_eigensystem,
    spgemm,
    normalized_laplacian_sym
)


jax.config.update("jax_enable_x64", True)

# more heavy duty
# 使用 Barabasi-Albert 模型生成一个规模更大的图
n_nodes = 10000  # 节点数
m_edges = 100      # 每个新节点连接到的现有节点数
print(f"正在生成一个包含 {n_nodes} 个节点和大约 {n_nodes * m_edges} 条边的 Barabasi-Albert 图...")

# 1. 创建 NetworkX 图对象
nx_g = nx.barabasi_albert_graph(n=n_nodes, m=m_edges, seed=42)
print("NetworkX 图已生成.")
print(f"节点数: {nx_g.number_of_nodes()}, 边数: {nx_g.number_of_edges()}")

# 2. 将 NetworkX 图转换为我们的 JAX Graph 对象
jax_g = from_networkx(nx_g)
print("JAX Graph 对象已创建.\n")

# --- 3. SpGEMM (消息传递) ---
print("--- 验证 SpGEMM ---")
# 创建一个随机的节点特征矩阵
key = jax.random.PRNGKey(42)
# 使用一个能被CPU核心数整除的特征维度，例如 128
n_features = 128
print(f"特征维度 = {n_features}")
features = jax.random.normal(key, (jax_g.n_nodes, n_features)) 

# --- NetworkX/SciPy SpGEMM (作为基准) ---
print("\n--- Scipy基准 ---")
import scipy.sparse
# 将 networkx 图转换为稀疏的 SciPy 矩阵
A_scipy = nx.to_scipy_sparse_array(nx_g, dtype=np.float32)
# 将 JAX 特征数组转换为 NumPy 数组
features_np = np.asarray(features)
# 预热
scipy_result = A_scipy @ features_np
# 性能比较
scipy_time = timeit.timeit(lambda: A_scipy @ features_np, number=10)
print(f"SciPy SpGEMM: {scipy_time:.6f}s")

# --- 常规 SpGEMM ---
print("\n--- 测试单线程 JAX SpGEMM ---")
# 我们的计算方式
spgemm_jax_result = spgemm(jax_g, features)
# 预热
spgemm_jax_result.block_until_ready()
# 性能比较
jax_time = timeit.timeit(lambda: spgemm(jax_g, features).block_until_ready(), number=10)
print(f"JAX SpGEMM速度: {jax_time:.6f}s")

print(f"JAX 结果是否与 SciPy 一致: {np.allclose(spgemm_jax_result, scipy_result, atol=1e-5)}")

# --- 并行 SpGEMM (pmap) ---
# 仅在有多个设备时运行
if jax.local_device_count() > 1:
    print(f"\n--- 测试并行 SpGEMM (pmap) on {jax.local_device_count()} devices ---")
    from kernels import spgemm_pmap # 导入 pmap 版本
    
    # 执行 pmap 版本的 spgemm
    spgemm_pmap_result = spgemm_pmap(jax_g, features)
    
    # 验证结果是否一致
    print(f"结果是否一致: {np.allclose(spgemm_jax_result, spgemm_pmap_result, atol=1e-5)}")

    # 性能比较
    # 预热
    spgemm_pmap_result.block_until_ready()
    pmap_time = timeit.timeit(lambda: spgemm_pmap(jax_g, features).block_until_ready(), number=10)
    
    print(f"并行 SpGEMM (pmap) 实现速度: {pmap_time:.6f}s")
    print(f"加速比: {jax_time / pmap_time:.2f}x")
    print(f"JAX 结果是否与 SciPy 一致: {np.allclose(spgemm_pmap_result, scipy_result, atol=1e-5)}")
else:
    print("\n--- 跳过 pmap 测试 (仅检测到单个设备) ---")

print("-" * 40)

正在生成一个包含 10000 个节点和大约 1000000 条边的 Barabasi-Albert 图...
NetworkX 图已生成.
节点数: 10000, 边数: 990000
JAX Graph 对象已创建.

--- 验证 SpGEMM ---
特征维度 = 128

--- Scipy基准 ---
SciPy SpGEMM: 0.391695s

--- 测试单线程 JAX SpGEMM ---
JAX SpGEMM速度: 1.767550s
JAX 结果是否与 SciPy 一致: True

--- 测试并行 SpGEMM (pmap) on 8 devices ---
结果是否一致: True
并行 SpGEMM (pmap) 实现速度: 1.138581s
加速比: 1.55x
JAX 结果是否与 SciPy 一致: True
----------------------------------------
